In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

import numpy as np
import sys
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers import LSTM
from keras.preprocessing.sequence import pad_sequences


Using TensorFlow backend.


In [2]:
data = pd.read_csv('intellisent/tweets_new.csv')
# Keeping only the neccessary columns
data.dtypes.index
data = data[['text', 'tweet_class']]

# data.drop(data['tweet_class'] == 2 , inplace=True)

#removing all Junk tweets
# data_all = data[ data.tweet_class != 2 ]

data_all = data.copy()

# print(len( data[data['tweet_class'] == 1]  ))

# print(len( data[data['tweet_class'] == -1]  ))

# print(len( data[data['tweet_class'] == 0]  ))

data = pd.DataFrame()

data = data.append(data_all[data_all['tweet_class']==-1])
data = data.append(data_all[data_all['tweet_class']==0][:125])
data = data.append(data_all[data_all['tweet_class']==1][:125])
data = data.append(data_all[data_all['tweet_class']==2][:125])

# data[data['tweet_class'] == -1]



In [3]:
data['text'] = data['text'].apply(lambda x: x.lower())
# data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

stop_words = set(stopwords.words('english'))

ps = PorterStemmer()

def clean(x):
    remove_url = re.sub(r'http\S+', '', x)
    remove_replies = re.sub(r'@\S+', '', remove_url)
    return remove_replies
    
data['cleaned'] = data['text'].apply(clean)

print(data['cleaned'])

# data["clean"] = data["unigrams"].apply(nltk.word_tokenize)


for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['cleaned'].values)
X = tokenizer.texts_to_sequences(data['cleaned'].values)
X = pad_sequences(X)

7      me gustó un video de    why people stopped wat...
12     i liked a  video   why people stopped watching...
19     the walking dead gotta be the dumbest show eve...
31     at least better than the walking dead aka 7 ep...
43     i’m sorry but this show is a shell of the walk...
63     series i once loved but had to bail on:\nlost\...
66      hey wanna fight since u guys are negan apolog...
78     i was dissapointed with everyone's reaction, t...
81                    when is the walking dead gonna end
96     i started binge watching the walking dead from...
116    fat people in theme parks are the real walking...
131    i cannot stress enough how much i hate the wal...
137                  the walking dead has got so boring 
140    carl when he finds out he how many views the w...
143    holy shit! please woman! this isn't a casting ...
144    is it me or is the walking dead been really bo...
147                                      i miss the old 
153     i've tried to make it w

In [4]:
embed_dim = 128
lstm_out = 196

# model = Sequential()
# model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1], dropout=0.2))
# model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
# model.add(Dense(4,activation='softmax'))
# model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model = Sequential()
model.add(Embedding(9000 + 1, 200, input_length = X.shape[1]))
model.add(Dropout(0.4))
model.add(LSTM(128))
model.add(Dense(64))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Dense(4))
model.add(Activation('sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 58, 200)           1800200   
_________________________________________________________________
dropout_1 (Dropout)          (None, 58, 200)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               168448    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 260       
__________

In [5]:
df = data.copy()
df.loc[df.tweet_class == -1, 'tweet_class'] = 3

Y = df['tweet_class']

# encoder = LabelBinarizer()
# transformed_Y = encoder.fit_transform(Y)

from keras.utils import to_categorical
transformed_Y = to_categorical(Y)

X_train, X_test, Y_train, Y_test = train_test_split(X,transformed_Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(335, 58) (335, 4)
(165, 58) (165, 4)


In [6]:
print(X_test.shape)
print(X_train.shape)
print(Y_test.shape)
print(Y_train.shape)

(165, 58)
(335, 58)
(165, 4)
(335, 4)


In [7]:
batch_size = 7
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
 - 16s - loss: 1.4468 - acc: 0.2746
Epoch 2/7
 - 17s - loss: 1.3896 - acc: 0.2746
Epoch 3/7
 - 17s - loss: 1.3215 - acc: 0.4119
Epoch 4/7
 - 18s - loss: 1.0158 - acc: 0.6030
Epoch 5/7
 - 17s - loss: 0.5820 - acc: 0.7672
Epoch 6/7
 - 16s - loss: 0.3531 - acc: 0.9015
Epoch 7/7
 - 17s - loss: 0.2242 - acc: 0.9582


In [11]:
validation_size = 50

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 1.61
acc: 0.54


In [12]:
my_test = tokenizer.texts_to_sequences(["the new season is bad bad bad"])

print(my_test) 

import numpy 

test1 = numpy.asarray(my_test)

test1 = pad_sequences(test1, maxlen=58)

test1

[[1, 98, 29, 8, 165, 165, 165]]


array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1,
         98,  29,   8, 165, 165, 165]], dtype=int32)

In [13]:
print(model.predict_classes(test1))

[0]
